In [222]:
import re
import requests
from bs4 import BeautifulSoup


In [135]:
text1 = input('Please enter the results of the OCR here:')

Please enter the results of the OCR here:ASYMMETRIK LTD Mike Smith Senior Software Engineer (410)555-1234 msmith@asymmetrik.com


In [136]:
text2 = input('Please enter the results of the OCR here:')

Please enter the results of the OCR here:Foobar Technologies Analytic Developer Lisa Haung 1234 Sentry Road Columbia, MD 12345 Phone: 410-555-1234 Fax: 410-555-4321 lisa.haung@foobartech.com


In [137]:
text3 = input('Please enter the results of the OCR here:')

Please enter the results of the OCR here:Arthur Wilson Software Engineer Decision & Security Technologies ABC Technologies 123 North 11th Street Suite 229 Arlington, VA 22209 Tel: +1 (703) 555-1259 Fax: +1 (703) 555-1200 awilson@abctech.com


In [133]:
def getPhoneNumber(text):
    #Writing regex to grab the first 5 characters before the phone number to ensure we do not grab a fax number
    phone_pattern = re.compile(r'((?:.{5})(?:\+1\s)?[\(]?\d{3}[\)|-]\s?\d{3}[-|\s]\d{4})')
    #matching the pattern and grabbing the second capture group which is the number
    number =re.findall(phone_pattern, text)
    #only selecting the numbers that are not labeled as a fax.
    number = [x[5:] for x in number if 'Fax' not in x][0]
    #cleaning the number by removing any non-digits
    number = re.sub(r'\D','',number)
    return('Phone: '+number)

'Phone: 17035551259'

In [143]:
def getEmail(text):
    #simple regex for email limiting the size of each component to 255 out of habit as regex's are expensive. 
    emailPattern = re.compile(r'\s((?:\w|\.){1,255}@\w{1,255}\.\w{1,255})')
    email = re.findall(emailPattern, text)
    return('Email: '+email[0].strip())

In [146]:
getEmail(text3)

'Email: awilson@abctech.com'

In [252]:
'''
I wanted to use a package like NLTK or SPACY to extact the names,
but both required specific large downloads and given that I cannot be sure of the system my code 
will be running on I implememted a solution using the Social Security Names.  I took a random guess and 
picked 1980 as the best year to compare for given names and then used a regex to extact the last name.  If 
this was going to be used in the wild I would want to download a more comprehensive name corpus or train an 
ML model
'''

last = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.all.last?#').text
first_f = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.female.first?#').text
first_m = requests.get('https://www2.census.gov/topics/genealogy/1990surnames/dist.male.first?#').text
first = ' '.join([first_f, first_m])
first = re.sub('\d',' ', first)

In [266]:
tokens = text3.split(' ')
for word in tokens:
    if (word.upper() in first) & :


Arthur
Wilson
VA


# Trying to Be Fancy

In [178]:
import nltk
!pip install nameparser
from nameparser.parser import HumanName

In [179]:
HumanName('ASYMMETRIK LTD Mike Smith Senior Software Engineer')

<HumanName : [
	title: '' 
	first: 'ASYMMETRIK' 
	middle: 'LTD Mike Smith Senior Software' 
	last: 'Engineer' 
	suffix: ''
	nickname: ''
]>

In [212]:
sentences = nltk.sent_tokenize(text1)
#sentences = [nltk.word_tokenize(sent.lower()) for sent in sentences]
sentences = [nltk.word_tokenize(sent) for sent in sentences]
sentences = [nltk.pos_tag(sent) for sent in sentences]
sentences

[[('ASYMMETRIK', 'NNP'),
  ('LTD', 'NNP'),
  ('Mike', 'NNP'),
  ('Smith', 'NNP'),
  ('Senior', 'NNP'),
  ('Software', 'NNP'),
  ('Engineer', 'NNP'),
  ('(', '('),
  ('410', 'CD'),
  (')', ')'),
  ('555-1234', 'JJ'),
  ('msmith', 'NN'),
  ('@', 'NNP'),
  ('asymmetrik.com', 'NN')]]

In [213]:
for tagged_sentence in sentences:
    for chunk in nltk.ne_chunk(tagged_sentence):
        print(type(chunk))
        if type(chunk) == nltk.tree.Tree:
            print(chunk, chunk.label())
            '''
            if chunk.label() == 'PERSON':
                print(chunk)
                '''

<class 'nltk.tree.Tree'>
(ORGANIZATION ASYMMETRIK/NNP) ORGANIZATION
<class 'nltk.tree.Tree'>
(ORGANIZATION
  LTD/NNP
  Mike/NNP
  Smith/NNP
  Senior/NNP
  Software/NNP
  Engineer/NNP) ORGANIZATION
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>
<class 'tuple'>


In [165]:
sentences

[[('Arthur', 'NNP'),
  ('Wilson', 'NNP'),
  ('Software', 'NNP'),
  ('Engineer', 'NNP'),
  ('Decision', 'NNP'),
  ('&', 'CC'),
  ('Security', 'NNP'),
  ('Technologies', 'NNPS'),
  ('ABC', 'NNP'),
  ('Technologies', 'NNPS'),
  ('123', 'CD'),
  ('North', 'NNP'),
  ('11th', 'CD'),
  ('Street', 'NNP'),
  ('Suite', 'NNP'),
  ('229', 'CD'),
  ('Arlington', 'NNP'),
  (',', ','),
  ('VA', 'NNP'),
  ('22209', 'CD'),
  ('Tel', 'NNP'),
  (':', ':'),
  ('+1', 'NN'),
  ('(', '('),
  ('703', 'CD'),
  (')', ')'),
  ('555-1259', 'JJ'),
  ('Fax', 'NN'),
  (':', ':'),
  ('+1', 'NN'),
  ('(', '('),
  ('703', 'CD'),
  (')', ')'),
  ('555-1200', 'CD'),
  ('awilson', 'NN'),
  ('@', 'CD'),
  ('abctech.com', 'NN')]]

In [216]:
!pip install spacy
from spacy.en import English
parser = English()

  Running setup.py bdist_wheel for ujson: started
  Running setup.py bdist_wheel for ujson: finished with status 'error'
  Complete output from command C:\Users\Kelly\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Kelly\\AppData\\Local\\Temp\\pip-install-bnqs0s6g\\ujson\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\Kelly\AppData\Local\Temp\pip-wheel-f17491sa --python-tag cp37:
  running bdist_wheel
  running build
  running build_ext
  building 'ujson' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": https://visualstudio.microsoft.com/downloads/
  
  ----------------------------------------
  Running setup.py clean for ujson
  Running setup.py bdist_wheel for dill: started
  Running setup.py bdist_wheel for dill: finished with status 'done'
  Stored in directory: C:\Users\Kelly\AppDat

  Failed building wheel for ujson
Command "C:\Users\Kelly\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\Kelly\\AppData\\Local\\Temp\\pip-install-bnqs0s6g\\ujson\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\Kelly\AppData\Local\Temp\pip-record-vjchpzjh\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\Kelly\AppData\Local\Temp\pip-install-bnqs0s6g\ujson\


ModuleNotFoundError: No module named 'spacy'

In [217]:
import spacy

ModuleNotFoundError: No module named 'spacy'